In [1]:
#Se importan las librerias
import os 
import pandas as pd
from epiweeks import Week


In [2]:
#Se lee el archivo
pdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
raw_dir = os.path.join(pdir,"data","raw")
data_dir = os.path.join(raw_dir,"dengue_data_all_municipalities.csv")
raw = pd.read_csv(data_dir)

In [3]:
#Columnas a transformar 
pop_cols = [x for x in raw.columns if "Population2" in x]
cases_cols = [x for x in raw.columns if "Cases2" in x]
temp_cols = [x for x in raw.columns if "TEMPERATURE" in x]
prec_cols = [x for x in raw.columns if "PRECIPITATION" in x]
weeks_cols = [x for x in raw.columns if "/w" in x]

In [4]:
# Se tranforman las columnas correspondientes de wide a long 
#--Population
df_pop = pd.melt(raw, id_vars=['Municipality code', 'Municipality'], value_vars=pop_cols,
        var_name='Year', value_name='Population')
df_pop["Year"] = df_pop.Year.str.strip("Population")
#raw.drop(columns=pop_cols, inplace=True)

#--Cases_year
df_cases_year = pd.melt(raw, id_vars=['Municipality code', 'Municipality'], value_vars=cases_cols,
        var_name='Year', value_name='Cases_per_Year')
df_cases_year["Year"] = df_cases_year.Year.str.strip("Cases")
raw.drop(columns=cases_cols, inplace=True)

#--Temperature
df_temp = pd.melt(raw, id_vars=['Municipality code', 'Municipality'], value_vars=temp_cols,
        var_name='Year_month', value_name='TEMPERATURE')
df_temp["Year_month"] = df_temp.Year_month.str.strip("TEMPERATURE_")
raw.drop(columns=temp_cols, inplace=True)

df_temp["Year"] = df_temp.Year_month.str.split("_").str[1]
df_temp["Month"] = df_temp.Year_month.str.split("_").str[0]
dict_months = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8, 'sep':9,
       'oct':10, 'nov':11, 'dec':12}
df_temp["Month_num"] = df_temp["Month"].replace(dict_months)
df_temp["Year"] ="20" + df_temp.Year
df_temp.drop(columns=["Year_month", "Month"], inplace = True)

#--Precipitation
df_prec = pd.melt(raw, id_vars=['Municipality code', 'Municipality'], value_vars=prec_cols,
        var_name='Year_month', value_name='PRECIPITATION')
df_prec["Year_month"] = df_prec.Year_month.str.strip("PRECIPITATION_")
raw.drop(columns=prec_cols, inplace=True)

df_prec["Year"] = df_prec.Year_month.str.split("_").str[1]
df_prec["Month"] = df_prec.Year_month.str.split("_").str[0]
dict_months = {'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8, 'sep':9,
       'oct':10, 'nov':11, 'dec':12}
df_prec["Month_num"] = df_prec["Month"].replace(dict_months)
df_prec["Year"] ="20" + df_prec.Year
df_prec.drop(columns=["Year_month", "Month"], inplace = True)

#-- Cases month
df_w = pd.melt(raw, id_vars=['Municipality code', 'Municipality'], value_vars=weeks_cols,
        var_name='Year_month', value_name='Cases')
df_w["week"] = df_w.Year_month.str.split("/w").str[1]
df_w["Year"] = df_w.Year_month.str.split("/w").str[0]
df_w['Year_month'] = df_w.Year_month.str.replace("/w","")
raw.drop(columns=weeks_cols, inplace=True)
df_w['week'] = df_w['Year_month'].apply(lambda x: Week.fromstring(x).enddate())
df_w['Month_num'] = pd.to_datetime(df_w['week']).dt.month
df_cases_month = df_w.groupby(["Municipality code",	"Municipality", "Year", "Month_num"]).sum().reset_index()

C:\Users\franc\AppData\Local\Temp\ipykernel_11940\696078585.py:51: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_cases_month = df_w.groupby(["Municipality code",	"Municipality", "Year", "Month_num"]).sum().reset_index()


In [5]:
df = pd.melt(raw, id_vars=raw.drop(columns=pop_cols).columns, value_vars=pop_cols,
        var_name='Year', value_name='Population')
df["Year"] = df_pop.Year.str.strip("Population")
df.drop(columns="Population", inplace=True)

In [6]:
#Se unen los dataframes en uno solo 
df_all = pd.merge(df_cases_month, df_prec, how="left", on=["Municipality code",	"Municipality", "Year", "Month_num"])
df_all = pd.merge(df_all, df_temp, how="left", on=["Municipality code",	"Municipality", "Year", "Month_num"])
df_all = pd.merge(df_all, df_pop, how="left", on=["Municipality code",	"Municipality", "Year"])
df_all = pd.merge(df_all, df, how="left", on=["Municipality code",	"Municipality", "Year"])

In [7]:
df_all.shape

(174876, 33)

In [8]:
trusted_dir = os.path.join(pdir,"data","trusted")

In [9]:
#Se exporta el nuevo dataset
df_all.to_csv(os.path.join(trusted_dir, "dengue_data.csv"), index=False)